In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import math
import time 
import operator
import yaml
import csv
import seaborn as sns
import pickle 

from sklearn.feature_selection import chi2, SelectKBest, VarianceThreshold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.base import TransformerMixin
from sklearn import metrics

## Import raw data

PARALYZED VETERANS OF AMERICA (PVA) data sample with ~95k people over ~22 months.

In [2]:
is_train = True # whether the procesing is for train data files or test (validation) data files

In [3]:
data_dir = './data/train/' if is_train else './data/test/'

raw_data_file = 'cup98LRN.csv' if is_train else 'cup98VAL.csv'

raw_data = pd.read_csv(data_dir + raw_data_file, sep = ',', error_bad_lines = False, 
                       low_memory = False, skip_blank_lines = True, na_values = [' '], 
                       keep_default_na = True, verbose = True)


if not is_train:
    # Need to merge target fields from `valtargt.csv` into `cup98VAL.csv` using the `CONTROLN` key.**
    target_data_file = 'valtargt.csv'
    target_data = pd.read_csv(data_dir + target_data_file, sep = ',', error_bad_lines = False, 
                       low_memory = False, skip_blank_lines = True, na_values = [' '], 
                       keep_default_na = True, verbose = True)

    targets = {} # { id: (TARGET_B, TARGET_D) }

    for _, row in target_data.iterrows():
        targets[row['CONTROLN']] = (row['TARGET_B'], row['TARGET_D'])
    
    target_Bs = []
    target_Ds = []

    for d_id in raw_data['CONTROLN']:
        target_Bs.append(targets[d_id][0])
        target_Ds.append(targets[d_id][1])

    raw_data['TARGET_B'] = target_Bs
    raw_data['TARGET_D'] = target_Ds

Tokenization took: 2916.37 ms
Type conversion took: 5955.19 ms
Parser memory cleanup took: 84.74 ms


In [4]:
#### Exploratory Analysis ####
print(raw_data.shape)
display(raw_data.head())

(95412, 481)


,ODATEDW,OSOURCE,TCODE,STATE,ZIP,MAILCODE,PVASTATE,DOB,NOEXCH,RECINHSE,...,TARGET_D,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,CLUSTER2,GEOCODE2
0,8901,GRI,0,IL,61081,NaN,NaN,3712,0,NaN,...,0.0,0,L,4,E,X,X,X,39.0,C
1,9401,BOA,1,CA,91326,NaN,NaN,5202,0,NaN,...,0.0,0,L,2,G,X,X,X,1.0,A
2,9001,AMH,1,NC,27017,NaN,NaN,0,0,NaN,...,0.0,1,L,4,E,X,X,X,60.0,C
3,8701,BRY,0,CA,95953,NaN,NaN,2801,0,NaN,...,0.0,1,L,4,E,X,X,X,41.0,C
4,8601,NaN,0,FL,33176,NaN,NaN,2001,0,X,...,0.0,1,L,2,F,X,X,X,26.0,A


### Helper functions

In [5]:
def get_range(df, feat):
    return str(len(Counter(df[df[feat].notna()][feat]))) + ' values from ' + \
            str(df[feat].min()) + ' to ' + str(df[feat].max())

In [6]:
def get_date(raw_date):
    """
    Returns the date in MM-YYYY format, and empty string ('') if input is nan.
    
    Sample input: get_date(9409.0) = '09-1994'
    """
    if math.isnan(raw_date):
        return ''
    
    month = int(raw_date % 100)
    year = int(raw_date // 100)
    return str(month).zfill(2) + '-19' + str(year)

def get_year(date):
    """
    Input: Dates in MM-YYYY format. Non null and non empty strings
    Output: int(YY)
    """
    if date == '':
        return None
    
    return int(date.split('-')[1][2:])

def get_month(date):
    """
    Input: Dates in MM-YYYY format. Non null and non empty strings
    Output: int(MM)
    """
    if date == '':
        return None
    
    return int(date.split('-')[0])

def months_diff(date_1, date_2, debug=True):
    """
    Returns num months difference, and None if one of the dates is invalid.
    Input: MM-YYYY and date_2 > date_1
    
    Sample input: months_diff('09-1994', '06-1995') = 9
    """
    if date_1 == '' or date_2 == '':
        return None
    if date_1 == None or date_2 == None:
        return None

    month_1, year_1 = date_1.split('-')
    month_1, year_1 = int(month_1), int(year_1)
    
    month_2, year_2 = date_2.split('-')    
    month_2, year_2 = int(month_2), int(year_2)

    if year_1 > year_2 or (year_1 == year_2 and month_1 > month_2): 
#         if debug:
#             print('Warning: date_1',  date_1, 'is larger than date_2', date_2) # raise Exception('date_1 is larger than date_2')
        return None

    num_months = 0
    if year_1 != year_2:
        num_months = (12 - month_1) + month_2
    else:
        num_months = month_2 - month_1
    
    assert num_months >= 0
    return num_months

def date_key(date):
    """
    Input: Dates in MM-YYYY format. Non null. Empty strings are ordered to be at the end.
    """
    if date == '':
        return 9999999 # larger than all other dates
    
    month, year = date.split('-')
    return int(year + month)

def compare_dates(date_1, date_2):
    """
    Input: Dates in MM-YYYY format. Non null and non empty strings
    Output: -1 if date_1 is smaller, 1 is date_1 is bigger, and 0 if equal
    """
    cmp = months_diff(date_1, date_2, debug=False)
    if cmp != None:
        return -1 if cmp > 0 else 0

    cmp = months_diff(date_2, date_1, debug=False)
    if cmp != None:
        return 1 if cmp > 0 else 0

    # If at least one date is invalid
    return -999

## Data Wrangling and EDA

In [ ]:
data = pd.DataFrame()

In [ ]:
# 0. Unique donor ID
data['id'] = raw_data['CONTROLN']

In [ ]:
# 1. 'age' - individual's age
data['age'] = raw_data['AGE']
print('Age range:', get_range(data, 'age'))

data['dob'] = raw_data['DOB']
print('DOB range:', get_range(data, 'dob'))

In [ ]:
# 2. 'income' - income bracket
data['income_bracket'] = raw_data['INCOME']
print('Income bracket range:', get_range(data, 'income_bracket'))

In [ ]:
# 2-24 
# ADATE_*: Date of mailing promotion.
# xxx RFA_*: Recency, Frequency, Amount
for i in range(2, 24 + 1):
    feat = 'ADATE_' + str(i)
    num_non_null = sum(raw_data[feat].isnull())
#     print(feat, 'null:', num_non_null, '(' + str(round(num_non_null / raw_data.shape[0] * 100, 2)) + '%)')
    print('Date of mailing promotion', feat, ':', get_range(raw_data, feat))

# 3-24
# RDATE_*: Date the gift was received
print()
for i in range(3, 24 + 1):
    feat = 'RDATE_' + str(i)
    num_non_null = sum(raw_data[feat].isnull())
#     print(feat, 'null:', num_non_null, '(' + str(round(num_non_null / raw_data.shape[0] * 100, 2)) + '%)')
    print('Date of receiving gift', feat, ':', get_range(raw_data, feat))

# 3-24
# RAMNT_*: Dollar amount of the gift 
print()
for i in range(3, 24 + 1):
    feat = 'RAMNT_' + str(i)
    num_non_null = sum(raw_data[feat].isnull())
#     print(feat, 'null:', num_non_null, '(' + str(round(num_non_null / raw_data.shape[0] * 100, 2)) + '%)')
#     print('Amount of received gift', feat, ':', get_range(raw_data, feat))

In [ ]:
# Train data-
# Promotion date range: 12-1993 (9312.0) to 06-1997 (9706)
# Gift donation date range: 09-1993 (9309.0) to 06-1998 (9806)

# Test data-
# Promotion date range: 09-1993 (9309.0) to 06-1997 (9706)
# Gift donation date range: 09-1993 (9309.0) to 06-1998 (9806)

# '97NK' ==> xxxx_2 (mailing was used to construct the target fields)
# '96NK' ==> xxxx_3
# '96TK' ==> xxxx_4
# '96SK' ==> xxxx_5
# '96LL' ==> xxxx_6
# '96G1' ==> xxxx_7
# '96GK' ==> xxxx_8
# '96CC' ==> xxxx_9
# '96WL' ==> xxxx_10
# '96X1' ==> xxxx_11
# '96XK' ==> xxxx_12
# '95FS' ==> xxxx_13
# '95NK' ==> xxxx_14
# '95TK' ==> xxxx_15
# '95LL' ==> xxxx_16
# '95G1' ==> xxxx_17
# '95GK' ==> xxxx_18
# '95CC' ==> xxxx_19
# '95WL' ==> xxxx_20
# '95X1' ==> xxxx_21
# '95XK' ==> xxxx_22
# '94FS' ==> xxxx_23
# '94NK' ==> xxxx_24

# Card promotions are promotion type- FS, NK, SK, TK, GK, XK (not used- UF, UU)
# Label/Calendar promotions are promotion type- LL, WL, CC, X1, G1


# `labels_only`
# LL mailings had labels only
# WL mailings had labels only

# `calendars_with_stickers`
# CC mailings are calendars with stickers but do not have labels

# `blank_cards_with_labels`
# FS mailings are blank cards that fold into thirds with labels
# NK mailings are blank cards with labels
# SK mailings are blank cards with labels

# `thank_you_with_labels`
# TK mailings have thank you printed on the outside with labels

# `greeting_cards_with_labels`
# GK mailings are general greeting cards (an assortment of birthday, sympathy, blank, & get well) with labels
# XK mailings are Christmas cards with labels

# `labels_and_notepad`
# X1 mailings have labels and a notepad
# G1 mailings have labels and a notepad

In [ ]:
promo_type = {
    # 1997
    2: 'blank_cards_with_labels', 
    # 1996
    3: 'blank_cards_with_labels',
    4: 'thank_you_with_labels',
    5: 'blank_cards_with_labels',
    6: 'labels_only',
    7: 'labels_and_notepad',
    8: 'greeting_cards_with_labels',
    9: 'calendars_with_stickers',
    10: 'labels_only',
    11: 'labels_and_notepad',
    12: 'greeting_cards_with_labels',
    13: 'blank_cards_with_labels',
    # 1995
    14: 'blank_cards_with_labels',
    15: 'thank_you_with_labels',
    16: 'labels_only',
    17: 'labels_and_notepad',
    18: 'greeting_cards_with_labels',
    19: 'calendars_with_stickers',
    20: 'labels_only',
    21: 'labels_and_notepad',
    22: 'greeting_cards_with_labels',
    # 1994
    23: 'blank_cards_with_labels',
    24: 'blank_cards_with_labels',
}

print(len(set(promo_type.values())), 'types of mailings:', set(promo_type.values()))

In [ ]:
END_DATE = '07-1998' # fake end date for the donor episode, for 'TARGET_D' donation gift

prom_dates = [] # Dates when promotions were sent for each donor
prom_types = [] # Type of promotion sent

gift_dates = [] # Dates when gifts were received from each donor for promotions sent
gift_amts = [] # Amounts when gifts were received from each donor for promotions sent

def get_amount(amt):
    return 0.0 if math.isnan(amt) else amt 

for i, row in raw_data.iterrows():
    if i % 10000 == 0:
        print('============================= at', i, 'of', len(raw_data), 'donors =============================')

    prom_dates.append([])
    prom_types.append([])

    gift_dates.append([])
    gift_amts.append([])
    
    # For 97NK (i=2), no gift date is available (but gift amount is in 'TARGET_D'). Add fake gift date
    for i in range(24, 2 - 1, -1):
        # Mailed promotion
        feat = 'ADATE_' + str(i)
        p_date = get_date(row[feat])
        if p_date == '':
            prom_dates[-1].append('')
        else:
            prom_dates[-1].append(p_date)
            prom_types[-1].append(promo_type[i])

        # Received gift
        if i == 2:
            g_amnt = get_amount(row['TARGET_D'])
            g_date = END_DATE if g_amnt > 0 else ''
        else:
            feat = 'RDATE_' + str(i)
            g_date = get_date(row[feat])
            
            feat = 'RAMNT_' + str(i)
            g_amnt = get_amount(row[feat])

        gift_dates[-1].append(g_date)
        gift_amts[-1].append(g_amnt)

    prom_dates[-1] = sorted(prom_dates[-1], key=date_key)
    prom_types[-1] = [t for _, t in sorted(zip(prom_dates[-1], prom_types[-1]), key=lambda pair: date_key(pair[0]))]
    
    # Use gift dates as sort key for amounts, before overwriting gifts dates with sorted dates
    gift_amts[-1] = [a for _, a in sorted(zip(gift_dates[-1], gift_amts[-1]), key=lambda pair: date_key(pair[0]))]

    gift_dates[-1] = sorted(gift_dates[-1], key=date_key)
    
    # TODO: FIXME: Change later when changing episodes to event wise ~~Sort acc. to sorted promotions order~~
#     gift_amts[-1] = [a for _, a in sorted(zip(prom_dates[-1], gift_amts[-1]), key=lambda pair: date_key(pair[0]))]

#     gift_dates[-1] = [gd for _, gd in sorted(zip(prom_dates[-1], gift_dates[-1]), key=lambda pair: date_key(pair[0]))]


data['prom_dates'] = prom_dates
data['prom_types'] = prom_types

data['gift_dates'] = gift_dates
data['gift_amts'] = gift_amts

print('Done!!!')

In [ ]:
# 3. 'num_gift_all' - number of gifts to date prior to final campaign
num_gifts_recorded = [sum([date != '' and compare_dates(date, '01-1997') == -1 for date in dates]) for dates in data['gift_dates']]
data['start_num_gift_all'] = raw_data['NGIFTALL'] - num_gifts_recorded
print('Total lifetime number of gifts given:', get_range(data, 'start_num_gift_all'))

In [ ]:
# 4. 'num_prom_all' - number of promotions to date prior to final campaign
# numprom in the original KDD Cup data takes on a single value for each individual, 
# and equals the total number of promotions mailed to that individual prior to the last campaign

# Initial value of num_prom_all- should be incremented with each promotion sent hereafter
num_proms_recorded = [sum([date != '' and compare_dates(date, '01-1997') == -1 for date in prom_dates]) for prom_dates in data['prom_dates']]
# num_proms_recorded = data['prom_dates'].map(len)
data['start_num_prom_all'] = raw_data['NUMPROM'] - num_proms_recorded
print('Total lifetime number of promotions sent:', get_range(data, 'start_num_prom_all'))

In [ ]:
# 5. 'frequency' - num_gift_all / num_prom_all
data['frequency'] = data['start_num_gift_all'] / data['start_num_prom_all']
print('Total lifetime frequency:', get_range(data, 'frequency'))

In [ ]:
# 8. 'amt_gift_all' - total amount of gifts to date

amnt_recorded = []
for i, dates in enumerate(data['gift_dates']):
    amnts = []
    for j, date in enumerate(dates):
        amnts.append(data['gift_amts'][i][j] if date != '' and compare_dates(date, '01-1997') == -1 else 0.0)
    amnt_recorded.append(sum(amnts))
        
data['amt_gift_all'] = raw_data['RAMNTALL'] - amnt_recorded
print('Total lifetime gift amount:', get_range(data, 'amt_gift_all'))

In [ ]:
# Date of sending first promotion to donor
first_prom = [min(dates, key=date_key) for dates in data['prom_dates']]
data['first_prom'] = first_prom

print('Date of sending first promotion:', str(len(Counter(data[data['first_prom'].notna()]['first_prom']))), \
      'values from', min(data['first_prom'], key=date_key), 'to', max(data['first_prom'], key=date_key))

In [ ]:
data.to_csv(data_dir + 'data_formatted_dates.csv', index=False)

pickle.dump(data, open(data_dir + "data_formatted_dates.p", "wb"))

In [ ]:
display(data)

# Make MDP data

This data set concerns direct mail promotions for soliciting donations, and contains demographic data as well as promotion history of 22 campaigns, conducted monthly over an approximately two year period.

For example, the feature named numprom in the original KDD Cup data takes on a single value for each individual, and equals the total number of promotions mailed to that individual prior to the last campaign. In our case, numprom is computed for each campaign by traversing the campaign history data backwards from the last campaign, subtracting one every time a promotion was mailed in a campaign.

Both the direct and indirect methods used as training data a subsample consisting of 10 thousand episodes, giving rise to 160 thousand event data. (The first 6 campaigns in each episode were discarded, due to lack of information regarding some of the temporal features.) 


- `TARGET_B`: Binary Indicator for Response to 97NK Mailing
- `TARGET_D`: Donation Amount (in $) associated with the Response to 97NK Mailing

http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.12.2474&rep=rep1&type=pdf

https://ieeexplore-ieee-org.libproxy1.nus.edu.sg/stamp/stamp.jsp?tp=&arnumber=1183879&tag=1

In [7]:
def get_age(dob, age, curr_date):
    return age

    # FIXME: correct to keep customer type constant?
    if dob == 0:
        return age    
    return months_diff(dob, curr_date) // 12

def get_past_6_month_dates(curr_date):
    """
    Input: MM-YYYY format
    Output: List of MM-YYYY dates just before curr_date, in increasing order
    """
    dates = []
    
    m = get_month(curr_date)    
    y = get_year(curr_date)
    while len(dates) < 6:
        m -= 1
        
        y = y - 1 if m == 0 else y
        m = 12 if m == 0 else m
        
        date = str(m).zfill(2) + '-19' + str(y).zfill(2)
        dates.insert(0, date)
    
    return dates

def get_most_recent_date(dates, curr_date):
    """
    Returns the date in `dates` that is closest to `curr_date` and occured before `curr_date`.
    Returns None if no value is found.
    
    Input: `dates` is a SORTED list of dates
    
    get_most_recent_date(['', '', '', '03-1995', '05-1995', '', '12-1995', '', ''], '08-1994') = None
    get_most_recent_date(['', '', '', '03-1995', '05-1995', '', '12-1995', '', ''], '07-1995') = '05-1995'
    """
    most_recent_date = None
    for d in dates:
        if d == '':
            continue
            
        if months_diff(d, curr_date) == None:
            # Since d is after curr_date, break out and return
            break

        most_recent_date = d
    
    return most_recent_date

def get_least_recent_date(dates, curr_date):
    """
    Returns the date in `dates` that is furthest from `curr_date` and occured before `curr_date`.
    Returns None if no value is found.
    
    Input: `dates` is a SORTED list of dates
    
    get_least_recent_date(['', '', '', '03-1995', '05-1995', '', '12-1995', '', ''], '08-1994') = None
    get_least_recent_date(['', '', '', '03-1995', '05-1995', '', '12-1995', '', ''], '07-1995') = '03-1995'
    """    
    for d in dates:
        if d == '':
            continue
        
        if months_diff(d, curr_date) == None:
            # Since d is after curr_date, break out and return
            break

        return d # least_recent_date
    
    return None

def get_first_date_after(dates, curr_date):
    """
    Returns the first date in `dates` that occurs after curr_date.
    Returns None if no value found.
    
    Input: `dates` is a SORTED list of dates
    
    get_first_date_after(['', '', '', '03-1995', '05-1995', '', '12-1995', '', ''], '05-1996') = None
    get_first_date_after(['', '', '', '03-1995', '05-1995', '', '12-1995', '', ''], '08-1994') = '03-1995'
    get_first_date_after(['', '', '', '03-1995', '05-1995', '', '12-1995', '', ''], '05-1995') = '05-1995'
    """
    for d in dates:
        if d == '':
            continue
        
        diff = months_diff(curr_date, d)
        if diff != None and diff >= 0:
            return d
    
    return None
    

def parse_dates_str(dates_str):
    """
    Returns a list of date strings from the input string dates.
    
    Example:
        Input: '[, , , 04-1995, , , 12-1995, , ]'
        Output: ['04-1995', '12-1995']
    """
    dates_str = dates_str[1:]
    dates_str = dates_str[:-1]
    dates_str = dates_str.strip()
    
    raw_dates = dates_str.split(',')
    
    dates = []
    for date in raw_dates:
        d = date.strip()
        if d != '':
            dates.append(d)
    
    return dates

In [8]:
# data = pd.read_csv(data_dir + 'data_formatted_dates.csv')

# data = pickle.load(open(data_dir + "data_formatted_dates.p", "rb"))

In [9]:
# Constants

EPISODE_START_DATE = '09-1994' # start of ADATE_18 (6 campaigns 19-24 are discarded); History that is not included in episode data
EPISODE_END_DATE = '08-1998' # fake end date, for 'TARGET_D' donation gift


# Value to use if timelag computed from available data is 0
NO_TIMELAG = 1

ACTION_NONE = 'no_action'

# TODO: reward
# FIXME: Question: Should reward for each promotion be given immediately (direct attribution), or month-wise

# http://kdd.ics.uci.edu/databases/kddcup98/epsilon_mirror/cup98doc.txt
# the package cost (including the mail cost) is $0.68 per piece mailed
PACKAGE_COST = 0.68 # Cost (negative reward) for taking each action


NaN = float('nan')

In [10]:
mdp_features = [
    'id', # 0.
    
    'reward',
    'bloc',
    'date',
    
    'age', # 1. individual's age
    'income_bracket', # 2. income bracket
    'num_gift_all', # 3. number of gifts to date
    'num_prom_all', # 4. number of promotions to date
    'frequency', # 5. num_gift_all / num_prom_all
    'gift_recency', # 6. number of months since last gift
    'last_gift', # 7. amount in dollars of last gift
    'amt_gift_all', # 8. total amount of gifts to date
    'num_rec_proms', # 9. num. of recent promotions (last 6 mo.)
    'num_rec_gifts', # 10. num. of recent gifts (last 6 mo.)
    'tot_rec_amt', # 11. total amount of recent gifts (6 mo.)
    'rec_amt_per_gift', # 12. recent gift amount per gift (6 mo.)
    'rec_amt_per_prom', # 13. recent gift amount per prom (6 mo.)
    'prom_recency', # 14. num. of months since last promotion
    'timelag', # 15. num. of mo’s from first prom to FIRST gift
    'gift_rec_ratio', # 16. gift_recency / timelag
    'prom_rec_ratio', # 17. prom_recency / timelag
    
# That is, this data set was used as input data set to various reinforcement learning methods, 
# except those features with ”*” in Table I, the mailedbits and respondedbits, were masked out.
    'responded_bit_1', # 18. whether responded last month
    'responded_bit_2', # 19. whether responded 2 months ago
    'responded_bit_3', # 20. whether responded 3 months ago
    'mailed_bit_1', # 21. whether promotion mailed last month
    'mailed_bit_2', # 22. whether promotion mailed 2 mo’s ago
    'mailed_bit_3', # 23. whether promotion mailed 3 mo’s ago

    'action', # 24. whether mailed in current promotion 
]

In [11]:
def get_initial_state(row, curr_date):
    state = {}

    state['id'] = row['id']
    # 1. individual's age
    state['age'] = get_age(row['dob'], row['age'], curr_date)
    # 2. income bracket
    state['income_bracket'] = row['income_bracket']
        
    # 3. number of gifts to date
    state['num_gift_all'] = row['start_num_gift_all']
    # 4. number of promotions to date
    state['num_prom_all'] = row['start_num_prom_all']
    # 5. num_gift_all / num_prom_all
    state['frequency'] = 0 if int(state['num_prom_all']) == 0 else state['num_gift_all'] / state['num_prom_all']

    g_dates = row['gift_dates'] 
    g_amts = row['gift_amts']
    # 6. number of months since last gift
    most_recent_date = get_most_recent_date(row['gift_dates'], curr_date)
    state['gift_recency'] = months_diff(most_recent_date, curr_date) if most_recent_date != None else NaN
    # 7. amount in dollars of last gift
    amts = [g_amts[i] for i, d in enumerate(g_dates) if d == most_recent_date]
    state['last_gift'] = max(amts) if len(amts) > 0 else 0.0 # FIXME: correct to choose max of donated gifts?
    # 8. total amount of gifts to date
    amts = []
    for i, d in enumerate(g_dates): 
        diff = months_diff(d, curr_date)
        if diff != None:
            amts.append(g_amts[i])
        else:
            break
    state['amt_gift_all'] = row['amt_gift_all'] + sum(amts) 
                
    num_rec_proms = 0
    num_rec_gifts = 0
    tot_rec_amt = 0    
    past_6_months = get_past_6_month_dates(curr_date)
    for d in past_6_months:
        num_rec_proms += row['prom_dates'].count(d)

        amts = [g_amts[i] for i, gd in enumerate(g_dates) if gd == d]
        num_rec_gifts += len(amts)
        tot_rec_amt += sum(amts)
        
    # 9. num. of recent promotions (last 6 mo.)
    state['num_rec_proms'] = num_rec_proms
    # 10. num. of recent gifts (last 6 mo.)
    state['num_rec_gifts'] = num_rec_gifts
    # 11. total amount of recent gifts (6 mo.)
    state['tot_rec_amt'] = tot_rec_amt
    # 12. recent gift amount per gift (6 mo.)
    state['rec_amt_per_gift'] = 0 if int(state['num_rec_gifts']) == 0 else state['tot_rec_amt'] / state['num_rec_gifts'] 
    # 13. recent gift amount per prom (6 mo.)
    state['rec_amt_per_prom'] = 0 if int(state['num_rec_proms']) == 0 else state['tot_rec_amt'] / state['num_rec_proms'] 
    # 14. num. of months since last promotion
    most_recent_date = get_most_recent_date(row['prom_dates'], curr_date)
    state['prom_recency'] = months_diff(most_recent_date, curr_date) if most_recent_date else NaN
    # 15. num. of mo’s from first prom to FIRST gift
    first_date_after = get_first_date_after(row['gift_dates'], row['first_prom'])
    timelag = months_diff(row['first_prom'], first_date_after)
    if timelag == 0: # cannot technically be 0, but we have missing data (prior to KDD Cup data)
        timelag = NO_TIMELAG
    state['timelag'] = timelag if timelag != None else NaN
    # 16. gift_recency / timelag
    state['gift_rec_ratio'] = NaN if math.isnan(state['timelag']) else state['gift_recency'] / state['timelag']
    # 17. prom_recency / timelag
    state['prom_rec_ratio'] = NaN if math.isnan(state['timelag']) else state['prom_recency'] / state['timelag']

    # 18. whether responded last month
    state['responded_bit_1'] = past_6_months[-1] in row['gift_dates']
    # 19. whether responded 2 months ago
    state['responded_bit_2'] = past_6_months[-2] in row['gift_dates']
    # 20. whether responded 3 months ago
    state['responded_bit_3'] = past_6_months[-3] in row['gift_dates']
    # 21. whether promotion mailed last month
    state['mailed_bit_1'] = past_6_months[-1] in row['prom_dates']
    # 22. whether promotion mailed 2 mo’s ago
    state['mailed_bit_2'] = past_6_months[-2] in row['prom_dates']
    # 23. whether promotion mailed 3 mo’s ago
    state['mailed_bit_3'] = past_6_months[-3] in row['prom_dates']
    
    # 24. whether mailed in current promotion  
    if curr_date in row['prom_dates']:
        i = -1
        i = row['prom_dates'].index(curr_date)
        state['action'] = row['prom_types'][i]
    else:
        state['action'] = ACTION_NONE

    state['reward'] = 0.0
    state['bloc'] = 0
    state['date'] = curr_date
    
    return state

In [12]:
TIME_PERIOD = 1 # number of months for which a data sample is generated

should_write = True
MDP_RAW_OLD_DATA_FILE = data_dir + 'mdp_raw_data(old_names)-' + str(TIME_PERIOD) + '_month_period.csv'

f = open(MDP_RAW_OLD_DATA_FILE, 'w', newline='')
w = csv.DictWriter(f, mdp_features)
if should_write:
    w.writeheader()
f.close()


start_time = time.time()
for i, row in data.iterrows():
    # Data rows for this donor
    episode = []

    curr_date = EPISODE_START_DATE    
    start_year = get_year(curr_date)
    end_year = get_year(EPISODE_END_DATE)

    episode.append(get_initial_state(row, curr_date))

    curr_tp = 0 # number of months since last data sample

    dates_6_months = [] # dates of the last 6 months in increasing order (MM-YYYY format)
    for year in range(start_year, end_year + 1):
        start_month = 1 if year != start_year else get_month(EPISODE_START_DATE)
        end_month = 12 if year != end_year else get_month(EPISODE_END_DATE)

        for month in range(start_month, end_month + 1):
            curr_tp += 1
            if curr_tp % TIME_PERIOD == 0:
                curr_tp = 0
            else:
                continue
            
            state = {}
            curr_date = get_date(int(str(year) + str(month).zfill(2)))

            state['id'] = row['id'] # 0.
            
            state['bloc'] = episode[-1]['bloc'] + 1
            state['date'] = curr_date

            # common- 1, 2
            state['age'] = get_age(row['dob'], row['age'], curr_date) # 1.
            state['income_bracket'] = row['income_bracket'] # 2.


            # gifts- 3, 6, 7, 8, 15
            g_dates = row['gift_dates'] 
            g_amts = row['gift_amts']
            
            timelag = episode[-1]['timelag']
            if curr_date in g_dates:
                state['num_gift_all'] = episode[-1]['num_gift_all'] + g_dates.count(curr_date) # 3.
                state['gift_recency'] = 0 # 6.

                amnts = [g_amts[i] for i, d in enumerate(g_dates) if d == curr_date]
                state['last_gift'] = max(amnts) # 7. # FIXME: correct to choose max of donated gifts?
                state['amt_gift_all'] = episode[-1]['amt_gift_all'] + sum(amnts) # 8.

                state['reward'] = sum(amnts)

                if math.isnan(timelag):
                    first_date_after = get_first_date_after(row['gift_dates'], row['first_prom'])
                    timelag = months_diff(row['first_prom'], first_date_after) 
                    if timelag == 0: # cannot technically be 0, but we have missing data (prior to KDD Cup data)
                        timelag = NO_TIMELAG
            else:
                state['num_gift_all'] = episode[-1]['num_gift_all'] # 3.
                state['gift_recency'] = episode[-1]['gift_recency'] + 1 # 6.
                state['last_gift'] = episode[-1]['last_gift'] # 7.
                state['amt_gift_all'] = episode[-1]['amt_gift_all'] # 8.
                
                state['reward'] = 0.0
            
            state['timelag'] = timelag if timelag else NaN # 15.

            # promotions- 4, 14, 24
            if curr_date in row['prom_dates']:
                num_proms = row['prom_dates'].count(curr_date)
                            
                state['num_prom_all'] = episode[-1]['num_prom_all'] + num_proms # 4.
                state['prom_recency'] = 0 # 14.

                # FIXME: assign cost based on promotion/action type
                state['reward'] -= PACKAGE_COST * num_proms

                # FIXME: multiple actions may occur if TIME_PERIOD is too large. 
                # Which one to choose? Add multiple rows? Or just pick one?
                idx = -1
                idx = row['prom_dates'].index(curr_date)
                state['action'] = row['prom_types'][idx] # 24.
            else:
                state['reward'] -= PACKAGE_COST * 0
                
                state['num_prom_all'] = episode[-1]['num_prom_all'] # 4.
                state['prom_recency'] = episode[-1]['prom_recency'] + 1 # 14.
                state['action'] = ACTION_NONE # 24.


            # history/misc- 5, 9, 10, 11, 12, 13 16, 17, 18, 19, 20, 21, 22, 23
            num_rec_proms = 0
            num_rec_gifts = 0
            tot_rec_amt = 0
            for d in dates_6_months:
                num_rec_proms += row['prom_dates'].count(d)

                amnts = [g_amts[i] for i, gd in enumerate(g_dates) if gd == d]
                num_rec_gifts += len(amnts)
                tot_rec_amt += sum(amnts)
                                  
            state['num_rec_proms'] = num_rec_proms # 9.
            state['num_rec_gifts'] = num_rec_gifts # 10.
            state['tot_rec_amt'] = tot_rec_amt # 11.

            state['frequency'] = 0 if int(state['num_prom_all']) == 0 else state['num_gift_all'] / state['num_prom_all'] # 5.
            state['rec_amt_per_gift'] = 0 if int(state['num_rec_gifts']) == 0 else state['tot_rec_amt'] / state['num_rec_gifts'] # 12.
            state['rec_amt_per_prom'] = 0 if int(state['num_rec_proms']) == 0 else state['tot_rec_amt'] / state['num_rec_proms'] # 13.
            state['gift_rec_ratio'] = NaN if math.isnan(state['timelag']) else state['gift_recency'] / state['timelag'] # 16.
            state['prom_rec_ratio'] = NaN if math.isnan(state['timelag']) else state['prom_recency'] / state['timelag'] # 17.

            state['responded_bit_1'] = len(dates_6_months) >= 1 and dates_6_months[-1] in g_dates # 18.
            state['responded_bit_2'] = len(dates_6_months) >= 2 and dates_6_months[-2] in g_dates # 19.
            state['responded_bit_3'] = len(dates_6_months) >= 3 and dates_6_months[-3] in g_dates # 20.

            state['mailed_bit_1'] = len(dates_6_months) >= 1 and dates_6_months[-1] in row['prom_dates'] # 21.
            state['mailed_bit_2'] = len(dates_6_months) >= 2 and dates_6_months[-2] in row['prom_dates'] # 21.
            state['mailed_bit_3'] = len(dates_6_months) >= 3 and dates_6_months[-3] in row['prom_dates'] # 21.

            # update dates
            dates_6_months.append(curr_date)
            if len(dates_6_months) > 6:
                dates_6_months.pop(0)

            episode.append(state)
    
    f = open(MDP_RAW_OLD_DATA_FILE, 'a', newline='')
    w = csv.DictWriter(f, mdp_features)
    if should_write:
        w.writerows(episode)
    f.close()
    
    if i % 5000 == 0:
        print('============================ Finished', i, 'donors ============================', (time.time() - start_time) / 60, 'minutes')
    
print('Done')

============================ Finished 0 donors ============================ 0.001379700501759847 minutes
============================ Finished 5000 donors ============================ 2.2068615436553953 minutes
============================ Finished 10000 donors ============================ 4.224872044722239 minutes
============================ Finished 15000 donors ============================ 6.2706930836041765 minutes
============================ Finished 20000 donors ============================ 8.212006103992461 minutes
============================ Finished 25000 donors ============================ 10.185337022940319 minutes
============================ Finished 30000 donors ============================ 12.275335109233856 minutes
============================ Finished 35000 donors ============================ 14.379203100999197 minutes
============================ Finished 40000 donors ============================ 16.46438846588135 minutes
============================ Finished 45000

In [13]:
# TIME_PERIOD = 2 
# MDP_RAW_OLD_DATA_FILE = data_dir + 'mdp_raw_data(old_names)-' + str(TIME_PERIOD) + '_month_period.csv'

mdp_data_old_names = pd.read_csv(MDP_RAW_OLD_DATA_FILE)

mdp_raw_data = pd.DataFrame()

mdp_raw_data['id'] = mdp_data_old_names['id']

mdp_raw_data['num_gifts_to_date'] = mdp_data_old_names['num_gift_all']
mdp_raw_data['num_promotions_to_date'] = mdp_data_old_names['num_prom_all']
mdp_raw_data['frequency-gifts_per_prom'] = mdp_data_old_names['frequency']
mdp_raw_data['last_gift_amount'] = mdp_data_old_names['last_gift']
mdp_raw_data['total_gifts_amount_to_date'] = mdp_data_old_names['amt_gift_all']
mdp_raw_data['num_recent_proms'] = mdp_data_old_names['num_rec_proms']
mdp_raw_data['num_recent_gifts'] = mdp_data_old_names['num_rec_gifts']
mdp_raw_data['total_recent_gifts_amount'] = mdp_data_old_names['tot_rec_amt']
mdp_raw_data['recent_amount_per_gift'] = mdp_data_old_names['rec_amt_per_gift']
mdp_raw_data['recent_amount_per_prom'] = mdp_data_old_names['rec_amt_per_prom']
mdp_raw_data['months_since_last_gift'] = mdp_data_old_names['gift_recency']
mdp_raw_data['months_since_last_prom'] = mdp_data_old_names['prom_recency']
mdp_raw_data['months_from_first_prom_to_gift'] = mdp_data_old_names['timelag']
mdp_raw_data['gift_recency_ratio'] = mdp_data_old_names['gift_rec_ratio']
mdp_raw_data['prom_recency_ratio'] = mdp_data_old_names['prom_rec_ratio']
mdp_raw_data['did_receive_gift_1_months_ago'] = mdp_data_old_names['responded_bit_1']
mdp_raw_data['did_receive_gift_2_months_ago'] = mdp_data_old_names['responded_bit_2']
mdp_raw_data['did_receive_gift_3_months_ago'] = mdp_data_old_names['responded_bit_3']
mdp_raw_data['did_mail_prom_1_months_ago'] = mdp_data_old_names['mailed_bit_1']
mdp_raw_data['did_mail_prom_2_months_ago'] = mdp_data_old_names['mailed_bit_2']
mdp_raw_data['did_mail_prom_3_months_ago'] = mdp_data_old_names['mailed_bit_3']

mdp_raw_data['age'] = mdp_data_old_names['age']
mdp_raw_data['income_bracket'] = mdp_data_old_names['income_bracket']

mdp_raw_data['action'] = mdp_data_old_names['action']

mdp_raw_data['reward'] = mdp_data_old_names['reward']
mdp_raw_data['bloc'] = mdp_data_old_names['bloc']
mdp_raw_data['date'] = mdp_data_old_names['date']

# mdp_raw_data['type'] = mdp_data_old_names['type']
# mdp_raw_data['state'] = mdp_data_old_names['state']


MDP_RAW_DATA_FILE = data_dir + '/mdp_raw_data-' + str(TIME_PERIOD) + '_month_period.csv'
mdp_raw_data.to_csv(MDP_RAW_DATA_FILE, index=False)

display(mdp_raw_data)

,id,num_gifts_to_date,num_promotions_to_date,frequency-gifts_per_prom,last_gift_amount,total_gifts_amount_to_date,num_recent_proms,num_recent_gifts,total_recent_gifts_amount,recent_amount_per_gift,...,did_receive_gift_3_months_ago,did_mail_prom_1_months_ago,did_mail_prom_2_months_ago,did_mail_prom_3_months_ago,age,income_bracket,action,reward,bloc,date
0,95515,24,52,0.461538,11.0,187.0,2,2,20.0,10.0,...,True,False,True,True,60.0,NaN,greeting_cards_with_labels,0.00,0,09-1994
1,95515,24,53,0.452830,11.0,187.0,0,0,0.0,0.0,...,False,False,False,False,60.0,NaN,greeting_cards_with_labels,-0.68,1,09-1994
2,95515,24,54,0.444444,11.0,187.0,1,0,0.0,0.0,...,False,True,False,False,60.0,NaN,labels_and_notepad,-0.68,2,10-1994
3,95515,24,56,0.428571,11.0,187.0,2,0,0.0,0.0,...,False,True,True,False,60.0,NaN,labels_only,-1.36,3,11-1994
4,95515,24,56,0.428571,11.0,187.0,4,0,0.0,0.0,...,False,True,True,True,60.0,NaN,no_action,0.00,4,12-1994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4675183,185114,38,138,0.275362,200.0,3775.0,0,0,0.0,0.0,...,False,False,False,False,80.0,5.0,no_action,0.00,44,04-1998
4675184,185114,38,138,0.275362,200.0,3775.0,0,0,0.0,0.0,...,False,False,False,False,80.0,5.0,no_action,0.00,45,05-1998
4675185,185114,38,138,0.275362,200.0,3775.0,0,0,0.0,0.0,...,False,False,False,False,80.0,5.0,no_action,0.00,46,06-1998
4675186,185114,38,138,0.275362,200.0,3775.0,0,0,0.0,0.0,...,False,False,False,False,80.0,5.0,no_action,0.00,47,07-1998


## Imputation for missing values

Shape: 95412 rows x columns

Number of null values:
1. `age`- 23665 
2. `income_bracket`- 21286
3. `num_gift_all` - 0 
4. `num_prom_all` - 
5. `frequency` - 
6. `gift_recency` - 
7. `lastgift` - 
8. `amt_gift_all` - 
9. `num_rec_proms` - 
10. `num_rec_gifts` - 
11. `tot_rec_amt` - 
12. `recamtpergift` - 
13. `recamtpergift` - 
14. `prom_recency` - 
15. `timelag`  - 
16. `recency_ratio` - 
17. `prom_rec_ratio` - 
18. `responded_bit[1]*` - 
19. `responded_bit[2]*` - 
20. `responded_bit[3]*` - 
21. `mailed_bit[1]*` - 
22. `mailed_bit[2]*` - 
23. `mailed_bit[3]*` - 
24. `action` - 


In [ ]:
# TIME_PERIOD = 2
# MDP_RAW_DATA_FILE = data_dir + '/mdp_raw_data-' + str(TIME_PERIOD) + '_month_period.csv'

# mdp_raw_data = pd.read_csv(MDP_RAW_DATA_FILE)

In [ ]:
display(data)
print(data.columns)

In [ ]:
display(mdp_raw_data)
print(mdp_raw_data.columns)

### Donor type

In [14]:
unique_donors = mdp_raw_data[['id', 'age', 'income_bracket']].drop_duplicates(subset=['id']).reset_index(drop=True)

mean_age = int(unique_donors['age'].mean())
print('Imputing `age` with mean value:', mean_age)
unique_donors['age'] = unique_donors['age'].fillna(mean_age).astype(int)
mdp_raw_data['age'] = mdp_raw_data['age'].fillna(mean_age).astype(int)

mean_income_bracket = int(unique_donors['income_bracket'].mean())
print('Imputing `income_bracket` with mean value:', mean_income_bracket)
unique_donors['income_bracket'] = unique_donors['income_bracket'].fillna(mean_income_bracket).astype(int)
mdp_raw_data['income_bracket'] = mdp_raw_data['income_bracket'].fillna(mean_income_bracket).astype(int)

Imputing `age` with mean value: 61
Imputing `income_bracket` with mean value: 3


In [ ]:
# unique_donors.to_csv(data_dir + 'unique_donors.csv', index=False)

In [ ]:
# fig = plt.figure(figsize=(5,5))

# plt.scatter(unique_donors['age'], unique_donors['income_bracket'])

# sns.pairplot(unique_donors[['age', 'income_bracket']].sample(n=10000, random_state=42), palette='Spectral')

### Impute State features (Clustered)

In [15]:
max_timelag = int(mdp_raw_data['months_from_first_prom_to_gift'].max())
factor = 3
print('Imputing NaN `timelag` with max value x', factor, ':', max_timelag * factor)
mdp_raw_data['months_from_first_prom_to_gift'] = mdp_raw_data['months_from_first_prom_to_gift'].fillna(max_timelag * factor).astype(int)

print('Possible values for `timelag`:', mdp_raw_data['months_from_first_prom_to_gift'].unique())

Imputing NaN `timelag` with max value x 3 : 57
Possible values for `timelag`: [ 1 10  2 12  4  5  7 13 18 14 15  3 57 16  8 17 11  6  9 19]


In [16]:
max_gift_recency = int(mdp_raw_data['months_since_last_gift'].max())
factor = 2
print('Imputing NaN `gift_recency` with max value x', factor, ':', max_gift_recency * factor)
mdp_raw_data['months_since_last_gift'] = mdp_raw_data['months_since_last_gift'].fillna(max_gift_recency * factor).astype(int)

print('Possible values for `gift_recency`:', mdp_raw_data['months_since_last_gift'].unique())

Imputing NaN `gift_recency` with max value x 2 : 102
Possible values for `gift_recency`: [  1   2   3   4   5   6   7   0   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32 102  33  34
  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51]


In [17]:
max_prom_recency = int(mdp_raw_data['months_since_last_prom'].max())
factor = 2
print('Imputing first NaN `prom_recency` with max prom_recency x', factor, 'months:', max_prom_recency * factor)

initial_prom_recency =  max_prom_recency * factor # For imputation

prom_recency_list = []
curr_id = ''
for i, row in mdp_raw_data.iterrows():
    if math.isnan(row['months_since_last_prom']):
        if row['id'] != curr_id:
            curr_id = row['id']
            prom_rec = initial_prom_recency
        else:
            prom_rec = prom_recency_list[i - 1] + 1

        prom_recency_list.append(prom_rec)
    else:
        prom_recency_list.append(row['months_since_last_prom'])

mdp_raw_data['months_since_last_prom'] = prom_recency_list
mdp_raw_data['months_since_last_prom'] = mdp_raw_data['months_since_last_prom'].astype(int)

print('Possible values for `prom_recency`:', mdp_raw_data['months_since_last_prom'].unique())

Imputing first NaN `prom_recency` with max prom_recency x 2 months: 52
Possible values for `prom_recency`: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 52 53 54 55 56 57 58 59 60
 61 62 63 64 65 66 67 68 15 16 17 18 19 20 21 22 23 24 25 26 69 70 71]


In [18]:
mdp_raw_data['gift_recency_ratio'] = mdp_raw_data['months_since_last_gift'] / mdp_raw_data['months_from_first_prom_to_gift']
mdp_raw_data['prom_recency_ratio'] = mdp_raw_data['months_since_last_prom'] / mdp_raw_data['months_from_first_prom_to_gift']

print('Updated NaN `gift_rec_ratio` and `prom_rec_ratio` using imputed values')

Updated NaN `gift_rec_ratio` and `prom_rec_ratio` using imputed values


In [19]:
binary_columns = [
    'did_receive_gift_1_months_ago', 'did_receive_gift_2_months_ago', 'did_receive_gift_3_months_ago', 
    'did_mail_prom_1_months_ago', 'did_mail_prom_2_months_ago', 'did_mail_prom_3_months_ago'
]
mdp_raw_data[binary_columns] = mdp_raw_data[binary_columns].astype(int)

In [20]:
# Check number of NaN fields
for c in mdp_raw_data.columns:
    print(c,sum(mdp_raw_data[c].isna()))

id 0
num_gifts_to_date 0
num_promotions_to_date 0
frequency-gifts_per_prom 0
last_gift_amount 0
total_gifts_amount_to_date 0
num_recent_proms 0
num_recent_gifts 0
total_recent_gifts_amount 0
recent_amount_per_gift 0
recent_amount_per_prom 0
months_since_last_gift 0
months_since_last_prom 0
months_from_first_prom_to_gift 0
gift_recency_ratio 0
prom_recency_ratio 0
did_receive_gift_1_months_ago 0
did_receive_gift_2_months_ago 0
did_receive_gift_3_months_ago 0
did_mail_prom_1_months_ago 0
did_mail_prom_2_months_ago 0
did_mail_prom_3_months_ago 0
age 0
income_bracket 0
action 0
reward 0
bloc 0
date 0


In [21]:
MDP_DATA_TYPE_IMPUTED_FILE = data_dir + 'mdp_raw_data-imputed-' + str(TIME_PERIOD) + '_month_period.csv'
mdp_raw_data.to_csv(MDP_DATA_TYPE_IMPUTED_FILE, index=False)

In [22]:
display(mdp_raw_data)

,id,num_gifts_to_date,num_promotions_to_date,frequency-gifts_per_prom,last_gift_amount,total_gifts_amount_to_date,num_recent_proms,num_recent_gifts,total_recent_gifts_amount,recent_amount_per_gift,...,did_receive_gift_3_months_ago,did_mail_prom_1_months_ago,did_mail_prom_2_months_ago,did_mail_prom_3_months_ago,age,income_bracket,action,reward,bloc,date
0,95515,24,52,0.461538,11.0,187.0,2,2,20.0,10.0,...,1,0,1,1,60,3,greeting_cards_with_labels,0.00,0,09-1994
1,95515,24,53,0.452830,11.0,187.0,0,0,0.0,0.0,...,0,0,0,0,60,3,greeting_cards_with_labels,-0.68,1,09-1994
2,95515,24,54,0.444444,11.0,187.0,1,0,0.0,0.0,...,0,1,0,0,60,3,labels_and_notepad,-0.68,2,10-1994
3,95515,24,56,0.428571,11.0,187.0,2,0,0.0,0.0,...,0,1,1,0,60,3,labels_only,-1.36,3,11-1994
4,95515,24,56,0.428571,11.0,187.0,4,0,0.0,0.0,...,0,1,1,1,60,3,no_action,0.00,4,12-1994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4675183,185114,38,138,0.275362,200.0,3775.0,0,0,0.0,0.0,...,0,0,0,0,80,5,no_action,0.00,44,04-1998
4675184,185114,38,138,0.275362,200.0,3775.0,0,0,0.0,0.0,...,0,0,0,0,80,5,no_action,0.00,45,05-1998
4675185,185114,38,138,0.275362,200.0,3775.0,0,0,0.0,0.0,...,0,0,0,0,80,5,no_action,0.00,46,06-1998
4675186,185114,38,138,0.275362,200.0,3775.0,0,0,0.0,0.0,...,0,0,0,0,80,5,no_action,0.00,47,07-1998


## Format and Save data

In [23]:
state_features = [
    'num_gifts_to_date', 'num_promotions_to_date',
    'frequency-gifts_per_prom', 'last_gift_amount',
    'total_gifts_amount_to_date', 'num_recent_proms', 'num_recent_gifts',
    'total_recent_gifts_amount', 'recent_amount_per_gift',
    'recent_amount_per_prom', 'months_since_last_gift',
    'months_since_last_prom', 'months_from_first_prom_to_gift',
    'gift_recency_ratio', 'prom_recency_ratio',
    'did_receive_gift_1_months_ago', 'did_receive_gift_2_months_ago',
    'did_receive_gift_3_months_ago', 'did_mail_prom_1_months_ago',
    'did_mail_prom_2_months_ago', 'did_mail_prom_3_months_ago', 
]

type_features = [
    'age', 'income_bracket',
]

mdp_raw_data.to_csv(data_dir + 'mdp_raw_data-imputed-' + str(TIME_PERIOD) + '_month_period-state_features.csv', columns=state_features, index=False)
mdp_raw_data.to_csv(data_dir + 'mdp_raw_data-imputed-' + str(TIME_PERIOD) + '_month_period-type_and_state_features.csv', columns=type_features + state_features, index=False)